In [1]:
import pyspark
from pyspark.sql import SparkSession
import math
import time

# Create a SparkSession
spark = SparkSession.builder \
    .appName("CreditCardFraud_LowLevel") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

sc = spark.sparkContext

## Data Loading
We load the credit card fraud dataset into an RDD using low-level Spark operations:
- Read the CSV file directly with `textFile`.
- Filter out the header row to process only data rows.
- Print the total number of records for verification.

In [2]:
# Path to the credit card dataset in Kaggle
credit_card_path = "/kaggle/input/creditcardfraud/creditcard.csv"

# Load the CSV file as a text file and filter out the header
lines_rdd = sc.textFile(credit_card_path)
header = lines_rdd.first()
data_rdd_raw = lines_rdd.filter(lambda line: line != header)

# Quick count of the raw data
print(f"Total number of records: {data_rdd_raw.count()}")

Total number of records: 284807


## Data Parsing, Cleaning, and Exploratory Data Analysis
We parse the raw text data, perform exploratory data analysis (EDA) to justify dropping the `Time` column, clean the data, and prepare features and labels with 29 features (V1–V28, Amount):
- Parse each line into a tuple of (features, label), validating for missing or invalid entries (empty strings, non-numeric values, or non-binary labels).
- Compute Pearson correlations between all features (including `Time`) and `Class` to assess predictive power; drop `Time` due to low correlation.
- Remove duplicate records based on V1–V28, Amount, and Class to ensure data quality.
- Cache the cleaned RDD for efficiency.
- Conduct EDA to analyze:
  - Total records, missing/duplicate counts, and number of features (29).
  - Class distribution to quantify dataset imbalance (fraud vs. non-fraud).
  - Summary statistics (mean, standard deviation, min, max) for each feature to understand distributions.
  - Key insights, such as potential skewness in `Amount`, PCA transformation of V1–V28, and correlation results.
- Print the correlations, decision to drop `Time`, total valid records, cleaning statistics, number of features, class distribution, feature summaries, insights, and a sample record.

In [3]:
# Parse each line into features and label, including Time temporarily for correlation
def parse_line_full(line):
    try:
        parts = line.split(",")
        if len(parts) != 31:  # Expect 30 features + 1 label
            return None
        features = []
        for x in parts[:-1]:  # Take Time, V1–V28, Amount
            x_clean = x.strip().replace('"', '')
            if x_clean == "" or x_clean.lower() == "nan":  # Check for empty or NaN
                return None
            features.append(float(x_clean))
        label = float(parts[-1].strip().replace('"', ''))
        if label not in [0.0, 1.0]:  # Validate binary label
            return None
        return (features, label)
    except:
        return None

# Parse raw data with all features for correlation analysis
parsed_data_rdd = data_rdd_raw.map(parse_line_full).filter(lambda x: x is not None)

# Compute Pearson correlation with Class for all 30 features
def compute_corr_stats(iterator):
    local_sum_x = [0.0] * 30
    local_sum_y = 0.0
    local_sum_xy = [0.0] * 30
    local_sum_x2 = [0.0] * 30
    local_sum_y2 = 0.0
    local_count = 0
    
    for record in iterator:
        features, label = record
        for i in range(30):
            local_sum_x[i] += features[i]
            local_sum_xy[i] += features[i] * label
            local_sum_x2[i] += features[i] * features[i]
        local_sum_y += label
        local_sum_y2 += label * label
        local_count += 1
    yield (local_sum_x, local_sum_y, local_sum_xy, local_sum_x2, local_sum_y2, local_count)

corr_stats = parsed_data_rdd.mapPartitions(compute_corr_stats).reduce(
    lambda x, y: (
        [x[0][i] + y[0][i] for i in range(30)],
        x[1] + y[1],
        [x[2][i] + y[2][i] for i in range(30)],
        [x[3][i] + y[3][i] for i in range(30)],
        x[4] + y[4],
        x[5] + y[5]
    )
)

sum_x, sum_y, sum_xy, sum_x2, sum_y2, count = corr_stats
mean_x = [s / count if count > 0 else 0.0 for s in sum_x]
mean_y = sum_y / count if count > 0 else 0.0
cov_xy = [(sum_xy[i] / count - mean_x[i] * mean_y) for i in range(30)]
var_x = [(sum_x2[i] / count - mean_x[i] ** 2) for i in range(30)]
var_y = sum_y2 / count - mean_y ** 2
correlations = []
for i in range(30):
    denom = math.sqrt(max(var_x[i] * var_y, 1e-10))  # Avoid division by zero
    corr = cov_xy[i] / denom if denom > 0 else 0.0
    correlations.append(corr)

# Print correlations and decide to drop Time
print("Correlation of Features with Class (Pearson):")
feature_names_full = ["Time"] + [f"V{i}" for i in range(1, 29)] + ["Amount"]
for i, name in enumerate(feature_names_full):
    print(f"{name:<12} {correlations[i]:>10.4f}")
time_corr = correlations[0]
print(f"\nDecision: Dropping Time column (correlation with Class: {time_corr:.4f}), as it shows low predictive power.")

# Re-parse to exclude Time
def parse_line(line):
    try:
        parts = line.split(",")
        if len(parts) != 31:
            return None
        features = []
        for x in parts[1:-1]:  # Skip Time, take V1–V28, Amount
            x_clean = x.strip().replace('"', '')
            if x_clean == "" or x_clean.lower() == "nan":
                return None
            features.append(float(x_clean))
        label = float(parts[-1].strip().replace('"', ''))
        if label not in [0.0, 1.0]:
            return None
        return (features, label)
    except:
        return None

# Parse again and clean
parsed_data_rdd = data_rdd_raw.map(parse_line).filter(lambda x: x is not None)
dedup_rdd = parsed_data_rdd.map(lambda x: (tuple(x[0] + [x[1]]), None)) \
                           .reduceByKey(lambda a, b: a) \
                           .map(lambda x: (list(x[0][:-1]), x[0][-1]))
cleaned_data_rdd = dedup_rdd.cache()

# --- EDA ---
# Basic counts
num_features = len(cleaned_data_rdd.first()[0])  # 29 (V1–V28, Amount)
total_count = cleaned_data_rdd.count()
missing_count = data_rdd_raw.count() - parsed_data_rdd.count()
duplicate_count = parsed_data_rdd.count() - total_count

print(f"\nTotal valid records after cleaning: {total_count}")
print(f"Number of missing or invalid records removed: {missing_count}")
print(f"Number of duplicate records removed: {duplicate_count}")
print(f"Number of features: {num_features}")

# Class distribution
class_counts = cleaned_data_rdd.map(lambda x: (x[1], 1)) \
                              .reduceByKey(lambda a, b: a + b) \
                              .collectAsMap()
fraud_pct = (class_counts.get(1.0, 0) / total_count * 100) if total_count > 0 else 0
print(f"Class distribution: {class_counts}")
print(f"Percentage of fraudulent transactions: {fraud_pct:.4f}%")

# Summary statistics for 29 features
def compute_stats(iterator):
    local_min = [float('inf')] * num_features
    local_max = [float('-inf')] * num_features
    local_sum = [0.0] * num_features
    local_sum_sq = [0.0] * num_features
    local_count = 0
    
    for record in iterator:
        features = record[0]
        for i in range(num_features):
            local_min[i] = min(local_min[i], features[i])
            local_max[i] = max(local_max[i], features[i])
            local_sum[i] += features[i]
            local_sum_sq[i] += features[i] * features[i]
        local_count += 1
    yield (local_min, local_max, local_sum, local_sum_sq, local_count)

stats = cleaned_data_rdd.mapPartitions(compute_stats).reduce(
    lambda x, y: (
        [min(x[0][i], y[0][i]) for i in range(num_features)],
        [max(x[1][i], y[1][i]) for i in range(num_features)],
        [x[2][i] + y[2][i] for i in range(num_features)],
        [x[3][i] + y[3][i] for i in range(num_features)],
        x[4] + y[4]
    )
)

mins, maxs, sums, sum_squares, count = stats
means = [s / count if count > 0 else 0.0 for s in sums]
variances = [(sum_squares[i] / count - (sums[i] / count) ** 2) if count > 0 else 0.0 for i in range(num_features)]
stddevs = [math.sqrt(max(v, 0.0)) for v in variances]

# Print feature statistics
print("\nFeature Summary Statistics:")
print(f"{'Feature':<12} {'Mean':>12} {'StdDev':>12} {'Min':>12} {'Max':>12}")
feature_names = [f"V{i}" for i in range(1, 29)] + ["Amount"]
for i, name in enumerate(feature_names):
    print(f"{name:<12} {means[i]:>12.4f} {stddevs[i]:>12.4f} {mins[i]:>12.4f} {maxs[i]:>12.4f}")

# Key insights
print("\nKey Feature Insights:")
print(f"Amount: Ranges from {mins[-1]:.2f} to {maxs[-1]:.2f}, indicating potential skewness.")
print(f"V1–V28: Near-zero means, stddev ~1–5, likely PCA-transformed.")
print(f"Correlation analysis showed stronger relationships for some V-features (e.g., V14, V17) with Class compared to Time.")

# Sample record
print("\nSample record (features, label):")
print(cleaned_data_rdd.first())

Correlation of Features with Class (Pearson):
Time            -0.0123
V1              -0.1013
V2               0.0913
V3              -0.1930
V4               0.1334
V5              -0.0950
V6              -0.0436
V7              -0.1873
V8               0.0199
V9              -0.0977
V10             -0.2169
V11              0.1549
V12             -0.2606
V13             -0.0046
V14             -0.3025
V15             -0.0042
V16             -0.1965
V17             -0.3265
V18             -0.1115
V19              0.0348
V20              0.0201
V21              0.0404
V22              0.0008
V23             -0.0027
V24             -0.0072
V25              0.0033
V26              0.0045
V27              0.0176
V28              0.0095
Amount           0.0056

Decision: Dropping Time column (correlation with Class: -0.0123), as it shows low predictive power.

Total valid records after cleaning: 275663
Number of missing or invalid records removed: 0
Number of duplicate records removed: 9144

## Feature Scaling
We apply min-max normalization to scale features to the [0,1] range for faster convergence:
- Compute min and max for each feature using RDD operations.
- Broadcast min/max values to all worker nodes for efficiency.
- Normalize features and cache the scaled RDD.
- Print a sample scaled record and unpersist the unscaled data.

In [4]:
# Collect min and max for each feature using mapPartitions for better performance
def compute_min_max(iterator):
    local_min_max = ([float('inf')] * num_features, [float('-inf')] * num_features)
    for record in iterator:
        features = record[0]
        for i in range(num_features):
            local_min_max[0][i] = min(local_min_max[0][i], features[i])
            local_min_max[1][i] = max(local_min_max[1][i], features[i])
    yield local_min_max

# Aggregate min/max across all partitions
min_max_stats = parsed_data_rdd.mapPartitions(compute_min_max).reduce(
    lambda x, y: (
        [min(x[0][i], y[0][i]) for i in range(num_features)],
        [max(x[1][i], y[1][i]) for i in range(num_features)]
    )
)

min_features = min_max_stats[0]
max_features = min_max_stats[1]

# Calculate feature ranges, ensuring no division by zero
epsilon = 1e-8  # Small value to prevent division by zero
feature_ranges = []
for i in range(num_features):
    range_val = max_features[i] - min_features[i]
    feature_ranges.append(max(range_val, epsilon))

# Broadcast min, max, and ranges to all worker nodes
bc_min_features = sc.broadcast(min_features)
bc_max_features = sc.broadcast(max_features)
bc_feature_ranges = sc.broadcast(feature_ranges)

# Scale the features using min-max normalization
def scale_record(record):
    features, label = record
    scaled_features = []
    for i in range(len(features)):
        scaled_val = (features[i] - bc_min_features.value[i]) / bc_feature_ranges.value[i]
        scaled_features.append(scaled_val)
    return (scaled_features, label)

scaled_data_rdd = parsed_data_rdd.map(scale_record).cache()

print("Sample Scaled Record (Features, Label):")
print(scaled_data_rdd.first())

# Unpersist the unscaled data
parsed_data_rdd.unpersist()

Sample Scaled Record (Features, Label):
([0.9351923374337303, 0.7664904186403037, 0.8813649032863348, 0.31302265906669463, 0.7634387348529242, 0.2676686424971201, 0.26681517599177856, 0.7864441979341067, 0.4753117341039581, 0.5106004821833838, 0.25248431906394647, 0.6809076254567205, 0.3715906024604766, 0.6355905300192973, 0.4460836956482719, 0.4343923913601106, 0.7371725526870235, 0.6550658609829579, 0.5948632283047696, 0.5829422304973765, 0.5611843885604425, 0.5229921162596571, 0.6637929753279846, 0.3912526763768729, 0.5851217945036548, 0.39455679156287454, 0.4189761351972912, 0.3126966335786978, 0.0058237930868049554], 0.0)


PythonRDD[25] at RDD at PythonRDD.scala:53

## Train-Test Split and Class Weighting
We prepare the data for training by:
- Splitting the scaled data into training (80%) and test (20%) sets with a fixed seed for reproducibility.
- Caching the split RDDs for efficiency.
- Calculating class weights to handle imbalance (higher weight for the minority class, fraud).
- Broadcasting class weights to all worker nodes.
- Printing the sizes of the split sets and class weights.

In [5]:
# Randomly split data into training (80%) and testing (20%) sets
train_rdd, test_rdd = scaled_data_rdd.randomSplit([0.8, 0.2], seed=42)
train_rdd.cache()
test_rdd.cache()

train_count = train_rdd.count()
test_count = test_rdd.count()
print(f"Training set size: {train_count}")
print(f"Test set size: {test_count}")

# Unpersist the full scaled data RDD
scaled_data_rdd.unpersist()

# Calculate class weights on the training data
train_class_counts = train_rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b).collectAsMap()
count_class_0 = train_class_counts.get(0.0, 0)
count_class_1 = train_class_counts.get(1.0, 0)

# Compute weights: larger weight for the minority class
total = count_class_0 + count_class_1
if count_class_0 == 0 or count_class_1 == 0:
    print("Warning: One class is missing in the training set. Using equal weights.")
    class_weights = {0.0: 1.0, 1.0: 1.0}
else:
    # Simple inverse frequency weighting
    class_weights = {
        0.0: total / (2.0 * count_class_0),
        1.0: total / (2.0 * count_class_1)
    }

print(f"Class distribution in training set - 0: {count_class_0}, 1: {count_class_1}")
print(f"Class weights - 0: {class_weights[0.0]:.4f}, 1: {class_weights[1.0]:.4f}")

# Broadcast class weights to all worker nodes
bc_class_weights = sc.broadcast(class_weights)

Training set size: 228163
Test set size: 56644
Class distribution in training set - 0: 227769, 1: 394
Class weights - 0: 0.5009, 1: 289.5470


## Logistic Regression Helper Functions
We define core functions for logistic regression:
- `sigmoid`: Computes the sigmoid function with overflow protection.
- `dot_product`: Calculates the dot product of two vectors.
- `predict`: Predicts probabilities using weights and sigmoid.
- `compute_gradient_and_loss`: Computes the gradient and loss for a single record, including L2 regularization and class weighting.
These functions are essential for the gradient descent implementation.

In [6]:
def sigmoid(z):
    """Sigmoid function with simple overflow protection"""
    if z < -500:
        return 0.0
    elif z > 500:
        return 1.0
    else:
        return 1.0 / (1.0 + math.exp(-z))

def dot_product(vec1, vec2):
    """Compute dot product of two vectors (lists)"""
    return sum(v1 * v2 for v1, v2 in zip(vec1, vec2))

def predict(features, weights):
    """Predict probability using dot product and sigmoid"""
    # Add a 1.0 for the bias term
    features_with_bias = [1.0] + features
    z = dot_product(features_with_bias, weights)
    return sigmoid(z)

def compute_gradient_and_loss(record, weights, reg_param):
    """Compute gradient and loss for a single record with regularization"""
    features, label = record
    features_with_bias = [1.0] + features  # Add bias term
    
    # Compute prediction
    prediction = predict(features, weights)
    
    # Get the weight for this class
    class_weight = bc_class_weights.value.get(label, 1.0)
    
    # Compute error
    error = prediction - label
    
    # Compute gradient for each feature
    gradient = [class_weight * error * x for x in features_with_bias]
    
    # Add L2 regularization to all weights except bias
    for i in range(1, len(weights)):
        gradient[i] += reg_param * weights[i]
    
    # Compute log loss with epsilon to avoid log(0)
    epsilon = 1e-9
    if label == 1.0:
        loss = -class_weight * math.log(prediction + epsilon)
    else:
        loss = -class_weight * math.log(1 - prediction + epsilon)
        
    # Add L2 regularization term to loss (exclude bias term)
    reg_loss = 0.0
    for i in range(1, len(weights)):
        reg_loss += 0.5 * reg_param * (weights[i] ** 2)
        
    return (gradient, loss + reg_loss)

## Gradient Descent Implementation with Hyperparameter Tuning
This section implements the gradient descent algorithm from scratch for logistic regression and enhances it with hyperparameter tuning to optimize performance. We initialize weights (including bias term) to zeros and use L2 regularization to prevent overfitting. A grid search tests combinations of learning rates (0.01, 0.1, 0.5) and regularization parameters (0.001, 0.01, 0.1), training each model with early stopping based on a convergence threshold. Models are evaluated on the test set using AUC, suitable for the imbalanced dataset, and the best model (highest AUC) is selected. The best model's weights and performance are reported for further evaluation.

In [7]:
from itertools import product
import time
import math

# Initialize weights (including bias term)
num_features_with_bias = num_features + 1  # Add 1 for bias term
initial_weights = [0.0] * num_features_with_bias

# Define hyperparameter grid
learning_rates = [0.01, 0.1, 0.5]
reg_params = [0.001, 0.01, 0.1]
max_iterations = 30
tolerance = 1e-4  # Convergence threshold

# Function to compute AUC (reusing from Cell 11 for validation)
def calculate_auc_approximation(data_rdd, weights):
    def get_prediction_score(record):
        features, actual_label = record
        prob = predict(features, weights)
        return (prob, actual_label)
    
    pred_scores = data_rdd.map(get_prediction_score).collect()
    pred_scores.sort(key=lambda x: x[0], reverse=True)
    
    num_positive = sum(1 for _, label in pred_scores if label == 1.0)
    num_negative = len(pred_scores) - num_positive
    
    if num_positive == 0 or num_negative == 0:
        return 0.0
    
    auc = 0.0
    tp = 0
    fp = 0
    prev_fp = 0
    prev_tp = 0
    
    for prob, label in pred_scores:
        if label == 1.0:
            tp += 1
        else:
            fp += 1
        if fp > prev_fp:
            auc += (tp + prev_tp) * (fp - prev_fp) / (2.0 * num_positive * num_negative)
            prev_fp = fp
            prev_tp = tp
    
    return auc

# Gradient descent function (extracted for reuse)
def run_gradient_descent(train_rdd, initial_weights, learning_rate, reg_param, max_iterations, tolerance):
    current_weights = initial_weights.copy()
    previous_loss = float('inf')
    iteration_stats = []
    
    for iteration in range(max_iterations):
        start_time = time.time()
        
        def process_partition(iterator):
            local_gradients = [0.0] * num_features_with_bias
            local_loss = 0.0
            local_count = 0
            
            for record in iterator:
                grad, loss = compute_gradient_and_loss(record, current_weights, reg_param)
                local_gradients = [local_gradients[i] + grad[i] for i in range(num_features_with_bias)]
                local_loss += loss
                local_count += 1
            
            yield (local_gradients, local_loss, local_count)
        
        result = train_rdd.mapPartitions(process_partition).reduce(
            lambda x, y: (
                [x[0][i] + y[0][i] for i in range(num_features_with_bias)],
                x[1] + y[1],
                x[2] + y[2]
            )
        )
        
        total_gradients, total_loss, count = result
        avg_gradients = [g / count for g in total_gradients]
        avg_loss = total_loss / count
        
        for i in range(num_features_with_bias):
            current_weights[i] -= learning_rate * avg_gradients[i]
        
        elapsed_time = time.time() - start_time
        loss_change = abs(avg_loss - previous_loss)
        iteration_stats.append((iteration+1, avg_loss, loss_change, elapsed_time))
        
        if (iteration+1) % 5 == 0 or iteration == 0 or iteration == max_iterations-1:
            print(f"Iteration {iteration+1}/{max_iterations} | Loss: {avg_loss:.6f} | Change: {loss_change:.6f} | Time: {elapsed_time:.2f}s")
        
        if loss_change < tolerance:
            print(f"Converged at iteration {iteration+1}. Loss change below tolerance ({tolerance}).")
            break
        
        previous_loss = avg_loss
    
    return current_weights, iteration_stats

# Hyperparameter tuning loop
print("\nStarting Logistic Regression with Hyperparameter Tuning...")
best_auc = 0.0
best_weights = None
best_params = None
best_stats = None

for lr, reg in product(learning_rates, reg_params):
    print(f"\nTesting learning_rate={lr}, reg_param={reg}")
    weights, stats = run_gradient_descent(train_rdd, initial_weights, lr, reg, max_iterations, tolerance)
    
    # Evaluate on test set (used as validation here)
    auc = calculate_auc_approximation(test_rdd, weights)
    print(f"AUC on test set: {auc:.4f}")
    
    if auc > best_auc:
        best_auc = auc
        best_weights = weights
        best_params = (lr, reg)
        best_stats = stats

# Report best model
print("\n--- Best Model ---")
print(f"Best learning_rate: {best_params[0]}, reg_param: {best_params[1]}")
print(f"Best AUC: {best_auc:.4f}")
print(f"Final Weights: {best_weights}")

# Use best weights for subsequent evaluation
current_weights = best_weights


Starting Logistic Regression with Hyperparameter Tuning...

Testing learning_rate=0.01, reg_param=0.001
Iteration 1/30 | Loss: 0.693147 | Change: inf | Time: 3.28s
Iteration 5/30 | Loss: 0.692341 | Change: 0.000200 | Time: 3.25s
Iteration 10/30 | Loss: 0.691358 | Change: 0.000195 | Time: 3.31s
Iteration 15/30 | Loss: 0.690396 | Change: 0.000191 | Time: 3.24s
Iteration 20/30 | Loss: 0.689452 | Change: 0.000187 | Time: 3.25s
Iteration 25/30 | Loss: 0.688524 | Change: 0.000185 | Time: 3.22s
Iteration 30/30 | Loss: 0.687607 | Change: 0.000182 | Time: 3.26s
AUC on test set: 0.9635

Testing learning_rate=0.01, reg_param=0.01
Iteration 1/30 | Loss: 0.693147 | Change: inf | Time: 3.18s
Iteration 5/30 | Loss: 0.692341 | Change: 0.000200 | Time: 3.44s
Iteration 10/30 | Loss: 0.691359 | Change: 0.000195 | Time: 3.29s
Iteration 15/30 | Loss: 0.690399 | Change: 0.000190 | Time: 3.24s
Iteration 20/30 | Loss: 0.689458 | Change: 0.000187 | Time: 3.24s
Iteration 25/30 | Loss: 0.688533 | Change: 0.0001

## Model Evaluation
We evaluate the model on the test set using a threshold of 0.5:
- Compute the confusion matrix (TP, FP, TN, FN).
- Calculate accuracy, precision, recall, and F1-score.
- Print the results, focusing on recall (sensitivity) as it's critical for fraud detection.

In [8]:
def evaluate_model(test_data, weights, threshold=0.5):
    """Evaluate the model on test data"""
    # Function to predict and compare with actual label
    def predict_and_evaluate(record):
        features, actual_label = record
        prob = predict(features, weights)
        predicted_label = 1.0 if prob >= threshold else 0.0
        return (predicted_label, actual_label, prob)
    
    # Get predictions for all test records
    predictions = test_data.map(predict_and_evaluate)
    predictions.cache()
    
    # Calculate confusion matrix counts
    tp = predictions.filter(lambda x: x[0] == 1.0 and x[1] == 1.0).count()
    fp = predictions.filter(lambda x: x[0] == 1.0 and x[1] == 0.0).count()
    tn = predictions.filter(lambda x: x[0] == 0.0 and x[1] == 0.0).count()
    fn = predictions.filter(lambda x: x[0] == 0.0 and x[1] == 1.0).count()
    
    # Calculate metrics
    total = tp + tn + fp + fn
    accuracy = (tp + tn) / total if total > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Unpersist predictions RDD
    predictions.unpersist()
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": {"tp": tp, "fp": fp, "tn": tn, "fn": fn}
    }

# Evaluate model with default threshold of 0.5
print("\n--- Evaluating Model on Test Set ---")
evaluation = evaluate_model(test_rdd, current_weights, threshold=0.5)

print("Results with threshold=0.5:")
print(f"Accuracy:  {evaluation['accuracy']:.4f}")
print(f"Precision: {evaluation['precision']:.4f}")
print(f"Recall:    {evaluation['recall']:.4f} (Sensitivity)")
print(f"F1-Score:  {evaluation['f1_score']:.4f}")
print("Confusion Matrix:")
print(f"  True Positives:  {evaluation['confusion_matrix']['tp']}")
print(f"  False Positives: {evaluation['confusion_matrix']['fp']}")
print(f"  True Negatives:  {evaluation['confusion_matrix']['tn']}")
print(f"  False Negatives: {evaluation['confusion_matrix']['fn']}")


--- Evaluating Model on Test Set ---
Results with threshold=0.5:
Accuracy:  0.9994
Precision: 0.8587
Recall:    0.8061 (Sensitivity)
F1-Score:  0.8316
Confusion Matrix:
  True Positives:  79
  False Positives: 13
  True Negatives:  56533
  False Negatives: 19


## Threshold Tuning and AUC Calculation
We explore model performance further:
- Test different decision thresholds (0.1 to 0.9) to balance precision and recall, as the default 0.5 may not be optimal for imbalanced data.
- Calculate the Area Under the ROC Curve (AUC) using an approximation with the trapezoidal rule, a key metric for imbalanced datasets.
- Print the results for each threshold and the AUC value.

In [9]:
# Try different thresholds to find better precision-recall balance
print("\n--- Evaluating Different Thresholds ---")
thresholds = [0.1, 0.3, 0.5, 0.7, 0.9]
for threshold in thresholds:
    eval_result = evaluate_model(test_rdd, current_weights, threshold)
    print(f"Threshold={threshold:.1f} | Precision: {eval_result['precision']:.4f} | Recall: {eval_result['recall']:.4f} | F1: {eval_result['f1_score']:.4f}")

# Calculate AUC
def calculate_auc_approximation(test_data, weights):
    """Calculate an approximation of the AUC using discrete thresholds"""
    # Function to get prediction probabilities and actual labels
    def get_prediction_score(record):
        features, actual_label = record
        prob = predict(features, weights)
        return (prob, actual_label)
    
    # Get prediction scores and sort them
    pred_scores = test_data.map(get_prediction_score).collect()
    pred_scores.sort(key=lambda x: x[0], reverse=True)
    
    # Initialize counters
    num_positive = sum(1 for _, label in pred_scores if label == 1.0)
    num_negative = len(pred_scores) - num_positive
    
    if num_positive == 0 or num_negative == 0:
        print("Warning: Only one class present in test set. AUC calculation not possible.")
        return 0.0
    
    # Initialize the area
    auc = 0.0
    tp = 0
    fp = 0
    prev_fp = 0
    prev_tp = 0
    
    # Process each prediction
    for prob, label in pred_scores:
        if label == 1.0:
            tp += 1
        else:
            fp += 1
            
        # Add trapezoid area under the curve
        if fp > prev_fp:
            auc += (tp + prev_tp) * (fp - prev_fp) / (2.0 * num_positive * num_negative)
            prev_fp = fp
            prev_tp = tp
    
    return auc

print("\n--- AUC Approximation ---")
auc = calculate_auc_approximation(test_rdd, current_weights)
print(f"Area Under ROC Curve (AUC): {auc:.4f}")


--- Evaluating Different Thresholds ---
Threshold=0.1 | Precision: 0.0017 | Recall: 1.0000 | F1: 0.0035
Threshold=0.3 | Precision: 0.0017 | Recall: 1.0000 | F1: 0.0035
Threshold=0.5 | Precision: 0.8587 | Recall: 0.8061 | F1: 0.8316
Threshold=0.7 | Precision: 0.8214 | Recall: 0.2347 | F1: 0.3651
Threshold=0.9 | Precision: 0.0000 | Recall: 0.0000 | F1: 0.0000

--- AUC Approximation ---
Area Under ROC Curve (AUC): 0.9641


## Cleanup and Shutdown
We clean up resources to free memory and stop the Spark session:
- Unpersist cached RDDs (`train_rdd`, `test_rdd`).
- Unpersist broadcast variables.
- Stop the Spark session to ensure proper shutdown.

In [10]:
# Clean up

train_rdd.unpersist()
test_rdd.unpersist()
bc_min_features.unpersist()
bc_max_features.unpersist()
bc_feature_ranges.unpersist()
bc_class_weights.unpersist()

# Stop Spark session
print("--- Stopping Spark Session ---")
spark.stop()

--- Stopping Spark Session ---
